In [2]:
!python3 --version

Python 3.6.9


In [3]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [41]:
import tensorflow as tf
import datetime

In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 6s 105us/sample - loss: 0.2212 - accuracy: 0.9342 - val_loss: 0.1075 - val_accuracy: 0.9678
Epoch 2/5
60000/60000 [==============================] - 6s 99us/sample - loss: 0.0973 - accuracy: 0.9708 - val_loss: 0.0757 - val_accuracy: 0.9775
Epoch 3/5
60000/60000 [==============================] - 6s 99us/sample - loss: 0.0688 - accuracy: 0.9784 - val_loss: 0.0657 - val_accuracy: 0.9782
Epoch 4/5
60000/60000 [==============================] - 6s 99us/sample - loss: 0.0537 - accuracy: 0.9822 - val_loss: 0.0758 - val_accuracy: 0.9764
Epoch 5/5
60000/60000 [==============================] - 6s 100us/sample - loss: 0.0439 - accuracy: 0.9858 - val_loss: 0.0593 - val_accuracy: 0.9828


In [7]:
%tensorboard --logdir logs/fit

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [8]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [10]:
!/opt/conda/bin/tensorboard --logdir logs/fit --bind_all

2020-03-25 02:17:18.889701: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-03-25 02:17:18.889830: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-03-25 02:17:18.889862: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
TensorBoard 2.1.1 at http://ip-10-133-135-70.ec2.internal:6006/ (Press CTRL+C to quit)
^C


In [7]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
from bs4 import BeautifulSoup
import plotly.graph_objs as go
import chart_studio.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

In [8]:
from pyathena import connect
import pandas as pd

conn = connect(s3_staging_dir='s3://freddie-mac-environments-jupyter-notebooks/jupyter/f404466',
region_name='us-east-1')

df = pd.read_sql("SELECT * FROM f354041.textmining limit 15000;", conn)

In [9]:
df.columns = ['label','feature']

In [10]:
df

,label,feature
0,compliancefee,natural hazards disclosure
1,titleclosingfee,title - settlement closing
2,taxotherfee,seller transfer tax
3,lenderotherfee,title - title underwriting fee
4,titleotherfee,title - title underwriting fee
...,...,...
14995,titleclosingprotectionletterfee,cpl fee
14996,escrowfee,escrowfee
14997,other,marketing fee
14998,surveyfee,survey


In [11]:
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
df['feature'] = df['feature'].apply(lambda x: BAD_SYMBOLS_RE.sub('', x))

In [13]:
df

,label,feature
0,compliancefee,natural hazards disclosure
1,titleclosingfee,title settlement closing
2,taxotherfee,seller transfer tax
3,lenderotherfee,title title underwriting fee
4,titleotherfee,title title underwriting fee
...,...,...
14995,titleclosingprotectionletterfee,cpl fee
14996,escrowfee,escrowfee
14997,other,marketing fee
14998,surveyfee,survey


In [17]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 10
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['feature'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 1444 unique tokens.


In [35]:
X = tokenizer.texts_to_sequences(df['feature'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (15000, 10)


In [36]:
Y = pd.get_dummies(df['label']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (15000, 50)


In [37]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(13500, 10) (13500, 50)
(1500, 10) (1500, 50)


In [50]:
!rm -rf ./logs/ 

In [48]:
def create_model2():
  return tf.keras.models.Sequential([
    tf.keras.layers.Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]),
    tf.keras.layers.SpatialDropout1D(0.2),
    tf.keras.layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.Dense(50, activation='softmax')
  ])

In [ ]:
# model = create_model2()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=X_train, 
          y=Y_train, 
          epochs=5, 
          validation_data=(X_test, Y_test), 
          callbacks=[tensorboard_callback])

In [55]:
!/opt/conda/bin/tensorboard --logdir logs/fit --bind_all

2020-03-25 15:43:26.657605: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-03-25 15:43:26.657714: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-03-25 15:43:26.657738: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
TensorBoard 2.1.1 at http://ip-10-133-135-70.ec2.internal:6006/ (Press CTRL+C to quit)
^C
